# RandomizedProgressiveHedging Project - Quick start

This section aims provides an explanation of how to build and solve a problem using RandomizedProgressiveHedging.jl by solving a toy problem. The equivalent script and ijulia notebook can be found in the `example` folder.

#### Installation
RandomizedProgressiveHedging.jl is a pure julia package. It can be installed from julia by using the built-in package manager:

In [1]:
using Pkg
Pkg.add("RandomizedProgressiveHedging")

 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


#### Getting solvers
RandomizedProgressiveHedging depends on other solvers to optimize the subproblems. All solvers interfaced with JuMP, the julia mathematical programming language, can be used in RandomizedProgressiveHedging, a list of which can be found at [JuMP's documentation](http://www.juliaopt.org/JuMP.jl/v0.19.0/installation/#Getting-Solvers-1).

Note that all algorithms layout subproblem with quadratic objectives. Default subproblem solver is the interior point algorithm Ipopt.

In [2]:
using Distributed
workers() == Vector([1]) && addprocs(3)     # add 3 workers besides master

@everywhere using JuMP, RandomizedProgressiveHedging
using DataStructures
workers()

3-element Array{Int64,1}:
 2
 3
 4

## Laying out a problem
We take the following problem as example:

\begin{aligned}
\underset{x}{\text{minimize}}\quad
& \sum_{t=1}^T C e_t + y_t \\
\text{s.t.}\quad
& q_t, y_t, e_t \ge 0 \\
& q_t \le W \\
& e_t+y_t \ge D \\
& q_1 = \bar{r}-y_1 \\
& q_t = q_{t-1}+r[\xi_t]-y_t, \; t = 2, \ldots, T.
\end{aligned}


where $C = 5$, $W = 8$, $D = 6$, $r = [2, 10]$. A scenario is defined by $(\xi_t)_{t=2, \ldots, T}$, for $\xi_t\in\{1,2\}$.

### Representing a scenario

A scenario is represented by the following structure:

In [3]:
@everywhere struct HydroThermalScenario <: AbstractScenario
    weather::Vector{Int}
end

Here, the attribut `weather` will hold one realisation of $(\xi_t)_{t=2, \ldots, T}$.

Along with this scenario structure, the function laying out the scenario objective function $f_s$ needs to be defined.
It takes as input the JuMP model that will hold $f_s$, an instance of the previously defined scenario structure, and the identifier of the scenario. 

In [4]:
@everywhere function build_fs!(model::JuMP.Model, s::HydroThermalScenario, id_scen::ScenarioId)
    C = 5
    W = 8
    D = 6
    rain = [2, 10]

    T = length(s.weather)+1
    Y = @variable(model, [1:3*T], base_name="y_s$id_scen")

    q = [Y[1+3*k] for k in 0:T-1]
    y = [Y[2+3*k] for k in 0:T-1]
    e = [Y[3+3*k] for k in 0:T-1]

    ## State variables constraints
    @constraint(model, Y[:] .>= 0)      # positivity constraint
    @constraint(model, q .<= W)         # reservoir max capacity
    @constraint(model, e .+ y .>= D)    # meet demand
    
    ## Dynamic constraints
    @constraint(model, q[1] == sum(rain)/length(rain) - y[1])
    @constraint(model, [t=2:T], q[t] == q[t-1] - y[t] + rain[s.weather[t-1]+1])
    
    objexpr = C*sum(e) + sum(y)

    return Y, objexpr, []
end


**Note**: The last item returned by the function should be the reference of constraints used to build the objective, none here. Such constraints can appear for example when modelling a ``max(u, v)`` in the objective as a variable ``m``, under the linear constraints ``m > u`` and ``m > v``.


### Representing the scenario tree
The scenario tree represents the stage up to which scenarios are equal.

If the problem scenario tree is a [perfect $m$-ary tree](https://en.wikipedia.org/wiki/M-ary_tree#Types_of_m-ary_trees) of depth $T$, it can be built using a buit-in function:
```julia
scenariotree = ScenarioTree(; depth=T, nbranching=m)
```

If the tree is not regular, or quite simple, it can be built by writing specifically the partition of equivalent scenarios per stage. A simple exmaple would be:
```julia
stageid_to_scenpart = [
    OrderedSet([BitSet(1:3)]),                      # Stage 1
    OrderedSet([BitSet(1), BitSet(2:3)]),           # Stage 2
    OrderedSet([BitSet(1), BitSet(2), BitSet(3)]),  # Stage 3
]
```
However this method is not efficient, and whenever possible, builtin functions should be priviledged.

### Building the `Problem`


In [5]:
scenid_to_weather(scen_id, T) = return [mod(floor(Int, scen_id / 2^i), 2) for i in T-1:-1:0]

T = 5
nbranching = 2

p = 0.5

nscenarios = 2^(T-1)
scenarios = HydroThermalScenario[ HydroThermalScenario( scenid_to_weather(scen_id, T-1) ) for scen_id in 0:nscenarios-1]
probas = [ prod(v*p + (1-v)*(1-p) for v in scenid_to_weather(scen_id, T-1)) for scen_id in 1:nscenarios ]

stage_to_dim = [3*i-2:3*i for i in 1:T]
scenariotree = ScenarioTree(; depth=T, nbranching=2)


pb = Problem(
    scenarios::Vector{HydroThermalScenario},
    build_fs!::Function,
    probas::Vector{Float64},
    nscenarios::Int,
    T::Int,
    stage_to_dim::Vector{UnitRange{Int}},
    scenariotree::ScenarioTree,
)

Multi-stage problem with:
 - #scenarios:   16
 - #stages   :   5
 - #dims     :   15


## Solving a problem

### Explicitly solving the problem

In [6]:
y_direct = solve_direct(pb)
println("\nDirect solve output is:")
display(y_direct)
@show objective_value(pb, y_direct);

--------------------------------------------------------
--- Direct solve
--------------------------------------------------------
Building global model...
Laying out nonanticipatory constraints...
Optimization... Done.
termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT


16×15 Array{Float64,2}:
 0.0  6.0  0.0  0.0  2.0  4.0  0.0  2.0  4.0  0.0   2.0  4.0  0.0   2.0  4.0
 0.0  6.0  0.0  0.0  2.0  4.0  0.0  2.0  4.0  0.0   2.0  4.0  4.0   6.0  0.0
 0.0  6.0  0.0  0.0  2.0  4.0  0.0  2.0  4.0  4.0   6.0  0.0  0.0   6.0  0.0
 0.0  6.0  0.0  0.0  2.0  4.0  0.0  2.0  4.0  4.0   6.0  0.0  8.0   6.0  0.0
 0.0  6.0  0.0  0.0  2.0  4.0  4.0  6.0  0.0  0.0   6.0  0.0  0.0   2.0  4.0
 0.0  6.0  0.0  0.0  2.0  4.0  4.0  6.0  0.0  0.0   6.0  0.0  4.0   6.0  0.0
 0.0  6.0  0.0  0.0  2.0  4.0  4.0  6.0  0.0  8.0   6.0  0.0  4.0   6.0  0.0
 0.0  6.0  0.0  0.0  2.0  4.0  4.0  6.0  0.0  8.0   6.0  0.0  8.0  10.0  0.0
 0.0  6.0  0.0  4.0  6.0  0.0  0.0  6.0  0.0  0.0   2.0  4.0  0.0   2.0  4.0
 0.0  6.0  0.0  4.0  6.0  0.0  0.0  6.0  0.0  0.0   2.0  4.0  4.0   6.0  0.0
 0.0  6.0  0.0  4.0  6.0  0.0  0.0  6.0  0.0  4.0   6.0  0.0  0.0   6.0  0.0
 0.0  6.0  0.0  4.0  6.0  0.0  0.0  6.0  0.0  4.0   6.0  0.0  8.0   6.0  0.0
 0.0  6.0  0.0  4.0  6.0  0.0  8.0  6.0  0.0  4.0   


Direct solve output is:

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

objective_value(pb, y_direct) = 50.0


### Solving with Progressive Hedging

In [7]:
y_PH = solve_progressivehedging(pb, printstep=5)
println("\nSequential solve output is:")
display(y_PH)
@show objective_value(pb, y_PH);


--------------------------------------------------------
--- Progressive Hedging
--------------------------------------------------------
Problem with:
 - nb scenarios       : 16
 - nb stages          : 5
 - variable dimension : 15
Algorithm parameters:
 - maxiter           1000  
 - maxtime (s)       3600  
 - ϵ_primal          0.0001
 - ϵ_dual            0.0001
 - μ                 3
 - maxcomputingtime  Inf
Initialisation... done
 it   primal res        dual res            dot(x,u)   objective
  0   3.0907561818e+00  3.4341735353e-01    2.998e-15  5.0668052421121914e+01


16×15 Array{Float64,2}:
 5.12297e-8  6.0  -7.7508e-9  -4.62562e-9  …  -9.37352e-9   2.0   4.0       
 5.12297e-8  6.0  -7.7508e-9  -4.62562e-9      4.0          6.0  -9.37352e-9
 5.12297e-8  6.0  -7.7508e-9  -4.62562e-9      8.83703e-8   6.0  -9.36378e-9
 5.12297e-8  6.0  -7.7508e-9  -4.62562e-9      8.0          6.0  -9.47389e-9
 5.12297e-8  6.0  -7.7508e-9  -4.62562e-9     -9.37352e-9   2.0   4.0       
 5.12297e-8  6.0  -7.7508e-9  -4.62562e-9  …   4.0          6.0  -9.37352e-9
 5.12297e-8  6.0  -7.7508e-9  -4.62562e-9      4.0          6.0  -9.39211e-9
 5.12297e-8  6.0  -7.7508e-9  -4.62562e-9      8.0         10.0  -9.49882e-9
 5.12297e-8  6.0  -7.7508e-9   4.0            -9.37352e-9   2.0   4.0       
 5.12297e-8  6.0  -7.7508e-9   4.0             4.0          6.0  -9.37352e-9
 5.12297e-8  6.0  -7.7508e-9   4.0         …   2.40668e-7   6.0  -9.37223e-9
 5.12297e-8  6.0  -7.7508e-9   4.0             8.0          6.0  -9.49152e-9
 5.12297e-8  6.0  -7.7508e-9   4.0            -9.073

  5   9.1240426327e-08  1.0137825142e-08   -1.443e-15  4.9999999119584594e+01
Computation time (s): 0.35979509353637695
Total time       (s): 3.577860116958618

Sequential solve output is:
objective_value(pb, y_PH) = 49.999999119584594


### Solving with Randomized Progressive Hedging

In [8]:
y_sync = solve_randomized_sync(pb, maxtime=5, printstep=50)
println("\nSynchronous solve output is:")
display(y_sync)
@show objective_value(pb, y_sync);


--------------------------------------------------------
--- Randomized Progressive Hedging - synchronous
--------------------------------------------------------
Problem with:
 - nb scenarios       : 16
 - nb stages          : 5
 - variable dimension : 15
Algorithm parameters:
 - maxiter           100000
 - maxtime (s)       5     
 - μ                 3.0
 - qdistr            pdistr
 - maxcomputingtime  Inf
Initialisation... done
   it   global residual   objective
    0   0.0000000000e+00  5.0668052421121914e+01
   50   8.8407947825e-01  4.9825855696971509e+01
  100   1.1936912310e-02  5.0023514088615414e+01
  150   4.7345020665e-04  5.0002150883595505e+01
  200   2.4037762164e-05  5.0000056936228397e+01
  250   1.5922290930e-05  5.0000003295716482e+01
  300   1.3937546562e-05  4.9999996495785211e+01
  350   6.7429314963e-08  4.9999999946795484e+01
  400   4.5564097222e-08  4.9999999895854017e+01
  450   5.0208543785e-06  4.9999998810499250e+01
  500   2.3465885038e-06  4.999999923

16×15 Array{Float64,2}:
 4.91164e-8  6.0  -6.5212e-9  1.13809e-6  …  -9.77273e-9   2.0   4.0       
 4.91164e-8  6.0  -6.5212e-9  1.13809e-6      4.0          6.0  -9.77273e-9
 4.91164e-8  6.0  -6.5212e-9  1.13809e-6      8.88241e-8   6.0  -9.37352e-9
 4.91164e-8  6.0  -6.5212e-9  1.13809e-6      8.0          6.0  -9.81164e-9
 4.91164e-8  6.0  -6.5212e-9  1.13809e-6     -9.37352e-9   2.0   4.0       
 4.91164e-8  6.0  -6.5212e-9  1.13809e-6  …   4.0          6.0  -9.37352e-9
 4.91164e-8  6.0  -6.5212e-9  1.13809e-6      4.0          6.0  -9.37352e-9
 4.91164e-8  6.0  -6.5212e-9  1.13809e-6      8.0         10.0  -9.49882e-9
 4.91164e-8  6.0  -6.5212e-9  4.0            -9.37352e-9   2.0   4.0       
 4.91164e-8  6.0  -6.5212e-9  4.0             4.0          6.0  -9.37352e-9
 4.91164e-8  6.0  -6.5212e-9  4.0         …   2.44516e-7   6.0  -9.37215e-9
 4.91164e-8  6.0  -6.5212e-9  4.0             8.0          6.0  -9.50201e-9
 4.91164e-8  6.0  -6.5212e-9  4.0            -8.33676e-9   6.0  

 834   2.2658540912e-06  4.9999998746245225e+01
Computation time (s): 4.606378555297852
Total time       (s): 5.02868390083313

Synchronous solve output is:
objective_value(pb, y_sync) = 49.999998746245225


### Solving with Parallel Randomized Progressive Hedging

Randomized parallel and asynchronous solves leverage the distributed capacities of julia. In order to be used, workers need to be available. Local or remote workers can be added with [`addprocs`](https://docs.julialang.org/en/v1/stdlib/Distributed/#Distributed.addprocs).

`RandomizedProgressiveHedging` and `JuMP` packages need to be available for all workers, along with the scenario object and objective build function.

In [11]:
y_par = solve_randomized_par(pb, maxtime=5, printstep=50)
println("\nSynchronous solve output is:")
display(y_par)
@show objective_value(pb, y_par);


--------------------------------------------------------
--- Randomized Progressive Hedging - parallel
--------------------------------------------------------
Problem with:
 - nb scenarios       : 16
 - nb stages          : 5
 - variable dimension : 15
Algorithm parameters:
 - maxiter           100000
 - maxtime (s)       5     
 - μ                 3.0
 - c                 0.9
 - qdistr            pdistr
 - maxcomputingtime  Inf
Available workers: 3
Initialisation... done
   it   residual            objective                
    0   5.7551589532e+01    5.0668052421121914e+01 
   50   7.8246397892e-03    4.9997024238698664e+01 
  100   3.7372150144e-07    4.9999999273131102e+01 
  150   1.5798015502e-07    4.9999999263151835e+01 
  200   4.0273569981e-07    4.9999999158948853e+01 
  250   7.6013432047e-07    4.9999999199599927e+01 
  300   2.1882324699e-06    4.9999999031925412e+01 
  350   9.3441350410e-07    4.9999999333984768e+01 
  400   2.4614527069e-06    4.9999999648003048e+01

16×15 Array{Float64,2}:
 4.56892e-8  6.0  -6.86111e-9  1.90229e-7  …  -9.37353e-9   2.0   4.0       
 4.56892e-8  6.0  -6.86111e-9  1.90229e-7      4.0          6.0  -9.37352e-9
 4.56892e-8  6.0  -6.86111e-9  1.90229e-7      8.8828e-8    6.0  -9.37352e-9
 4.56892e-8  6.0  -6.86111e-9  1.90229e-7      8.0          6.0  -9.45255e-9
 4.56892e-8  6.0  -6.86111e-9  1.90229e-7     -9.37352e-9   2.0   4.0       
 4.56892e-8  6.0  -6.86111e-9  1.90229e-7  …   4.0          6.0  -9.37352e-9
 4.56892e-8  6.0  -6.86111e-9  1.90229e-7      4.0          6.0  -9.37352e-9
 4.56892e-8  6.0  -6.86111e-9  1.90229e-7      8.0         10.0  -9.49884e-9
 4.56892e-8  6.0  -6.86111e-9  4.0            -9.37352e-9   2.0   4.0       
 4.56892e-8  6.0  -6.86111e-9  4.0             4.0          6.0  -9.37352e-9
 4.56892e-8  6.0  -6.86111e-9  4.0         …   2.4426e-7    6.0  -9.37209e-9
 4.56892e-8  6.0  -6.86111e-9  4.0             8.0          6.0  -9.50167e-9
 4.56892e-8  6.0  -6.86111e-9  4.0            -8.782

  673   7.6374488744e-07    4.9999999260651357e+01 
Computation time (s): 4.674056529998779
Total time       (s): 5.019078016281128

Synchronous solve output is:
objective_value(pb, y_par) = 49.99999926065136


### Solving with Asynchronous Randomized Progressive Hedging

In [10]:
y_async = solve_randomized_async(pb, maxtime=5, printstep=10)
println("Asynchronous solve output is:")
display(y_async)
@show objective_value(pb, y_async)


--------------------------------------------------------
--- Randomized Progressive Hedging - asynchronous
--------------------------------------------------------
Problem with:
 - nb scenarios       : 16
 - nb stages          : 5
 - variable dimension : 15
Algorithm parameters:
 - maxiter           100000
 - maxtime (s)       5     
 - μ                 3.0
 - c                 0.9
 - stepsize          nothing
 - qdistr            pdistr
 - maxcomputingtime  Inf
Available workers: 3
Initialisation... done
   it   residual            objective                 τ    delay
    0   0.0000000000e+00    5.0668052421121914e+01    4    0
 2398   7.2785595727e-05    5.0000060077658617e+01   38    6
Computation time (s): 4.497647047042847
Total time       (s): 5.034710168838501


16×15 Array{Float64,2}:
 6.81038e-5  5.99996  6.80457e-5  …  -9.43884e-9   2.00032   3.99968   
 6.81038e-5  5.99996  6.80457e-5      4.0          6.0      -9.37353e-9
 6.81038e-5  5.99996  6.80457e-5      8.86036e-8   6.0      -9.65906e-9
 6.81038e-5  5.99996  6.80457e-5      7.99999      6.0      -9.54338e-9
 6.81038e-5  5.99996  6.80457e-5     -9.37402e-9   2.00038   3.99962   
 6.81038e-5  5.99996  6.80457e-5  …   4.0          6.0      -9.37352e-9
 6.81038e-5  5.99996  6.80457e-5      3.99966      6.0      -9.3737e-9 
 6.81038e-5  5.99996  6.80457e-5      8.0          9.99919  -9.499e-9  
 6.81038e-5  5.99996  6.80457e-5     -9.37354e-9   2.0       4.0       
 6.81038e-5  5.99996  6.80457e-5      4.00002      6.00003  -9.37361e-9
 6.81038e-5  5.99996  6.80457e-5  …   2.42333e-7   6.0      -9.37221e-9
 6.81038e-5  5.99996  6.80457e-5      7.99999      6.0      -9.49872e-9
 6.81038e-5  5.99996  6.80457e-5     -7.67813e-9   6.0       1.86549e-6
 6.81038e-5  5.99996  6.80457e-5      7.

Asynchronous solve output is:
objective_value(pb, y_async) = 50.00006007765862


50.00006007765862